# Tutorial on Mass Simulations

In [1]:
from matplotlib import pyplot as plt
import poisson_approval as pa

## Probability Estimation

Probability that a TauVector drawn uniformly at random has a direct focus:

In [2]:
def test_direct_focus(tau):
    return tau.focus == pa.Focus.DIRECT

In [3]:
pa.probability(
    generator=pa.GeneratorTauVectorUniform(),
    n_samples=100,
    test=test_direct_focus
)

0.64

To avoid the definition of an auxiliary function, you can use a "lambda". For example:

In [4]:
pa.probability(
    generator=pa.GeneratorTauVectorUniform(),
    n_samples=100,
    test=lambda tau: tau.focus == pa.Focus.DIRECT
)

0.52

Probability that a ProfileOrdinal drawn uniformly at random has a direct equilibrium, conditionally on having a strict Condorcet winner:

In [5]:
def test_exists_direct_equilibrium(profile):
    return len(profile.analyzed_strategies.equilibria) > 0

In [6]:
def test_is_strictly_condorcet(profile):
    return profile.is_profile_condorcet == 1.0

In [7]:
# pa.probability(
#     generator=pa.GeneratorProfileOrdinalUniform(),
#     n_samples=100,
#     test=test_exists_direct_equilibrium,
#     conditional_on=test_is_strictly_condorcet
# )

Alternate syntax using lambdas:

In [8]:
# pa.probability(
#     generator=pa.GeneratorProfileOrdinalUniform(),
#     n_samples=100,
#     test=lambda profile: len(profile.analyzed_strategies.equilibria) > 0,
#     conditional_on=lambda profile: profile.is_profile_condorcet == 1.0
# )

## Grid explorations

### ExploreGridProfilesOrdinal

Here, we examine all examples with a given denominator in the profile. For example, if $\textrm{denominator} = 9$, we will examine profiles such as $(1/9, 2/9, 2/9, 1/9, 1/3, 0)$ or $(1/9, 5/9, 0, 3/9, 0, 0)$, etc.

Beforehand, we may define a test function that indicates what kinds of profiles we want to study. In this example, we restrict to Condorcet profiles that have at least a utility-dependent equilibrium.

In [9]:
def test_condorcet_with_utility_dependent(profile):
    return(
        profile.is_profile_condorcet == 1.0
        and len(profile.analyzed_strategies.utility_dependent) > 0
    )

N.B.: as in the example above, it is better to put the fast tests (here, is_profile_condorcet) before the slow tests (here, having no true equilibrium). It speeds up the program significantly.

Launch the exploration:

In [10]:
exploration = pa.ExploreGridProfilesOrdinal(
    denominator=3, test=test_condorcet_with_utility_dependent)
exploration

0 equilibrium, 1 utility-dependent equilibrium, 3 non-equilibria
<abc: 2/3, cab: 1/3> (Condorcet winner: a)

0 equilibrium, 1 utility-dependent equilibrium, 7 non-equilibria
<abc: 1/3, acb: 1/3, bac: 1/3> (Condorcet winner: a)

0 equilibrium, 2 utility-dependent equilibria, 2 non-equilibria
<abc: 2/3, cba: 1/3> (Condorcet winner: a)
<abc: 2/3, bca: 1/3> (Condorcet winner: a)

0 equilibrium, 3 utility-dependent equilibria, 5 non-equilibria
<abc: 1/3, acb: 1/3, bca: 1/3> (Condorcet winner: a)

2 equilibria, 1 utility-dependent equilibrium, 5 non-equilibria
<abc: 1/3, bac: 1/3, cab: 1/3> (Condorcet winner: a)

Access one profile in particular in order to study it:

In [11]:
profile = exploration[(0, 1, 3)][0]
profile

<abc: 2/3, cab: 1/3> (Condorcet winner: a)

If the test breaks the symmetry between candidates, you should use the option $\verb|standardized = False|$:

In [12]:
def test_a_is_condorcet_winner(profile):
    return 'a' in profile.condorcet_winners
exploration = pa.ExploreGridProfilesOrdinal(
    test=test_a_is_condorcet_winner, denominator=2, standardized=False)
exploration

0 equilibrium, 1 utility-dependent equilibrium, 3 non-equilibria
<bac: 1/2, cab: 1/2> (Condorcet winner: a, b, c)
<acb: 1/2, cba: 1/2> (Condorcet winner: a, c)
<acb: 1/2, bca: 1/2> (Condorcet winner: a, b, c)
<acb: 1/2, bac: 1/2> (Condorcet winner: a, b)
<abc: 1/2, cba: 1/2> (Condorcet winner: a, b, c)
<abc: 1/2, cab: 1/2> (Condorcet winner: a, c)
<abc: 1/2, bca: 1/2> (Condorcet winner: a, b)

1 equilibrium, 0 utility-dependent equilibrium, 1 non-equilibrium
<acb: 1> (Condorcet winner: a)
<abc: 1> (Condorcet winner: a)

1 equilibrium, 0 utility-dependent equilibrium, 3 non-equilibria
<acb: 1/2, cab: 1/2> (Condorcet winner: a, c)
<abc: 1/2, bac: 1/2> (Condorcet winner: a, b)
<abc: 1/2, acb: 1/2> (Condorcet winner: a)

### ExploreGridTaus

Define a test and explore:

In [13]:
def test_direct_focus(tau):
    return tau.focus == pa.Focus.DIRECT
exploration = pa.ExploreGridTaus(
    denominator=6, test=test_direct_focus)
exploration

<a: 1/3, ab: 1/6, bc: 1/2> ==> b
<a: 1/3, b: 1/6, bc: 1/2> ==> b
<a: 1/3, ab: 1/6, ac: 1/3, b: 1/6> ==> a
<a: 1/3, ac: 1/6, b: 1/3, c: 1/6> ==> a
<a: 1/2, ab: 1/3, ac: 1/6> ==> a
<a: 1/2, ac: 1/3, b: 1/6> ==> a
<a: 1/2, ab: 1/6, b: 1/6, c: 1/6> ==> a

Access one tau in particular in order to study it:

In [14]:
tau = exploration[0]
tau

<a: 1/3, ab: 1/6, bc: 1/2> ==> b

## Meta-Analysis: NiceStatsProfileOrdinal

Define a protocol of meta-analysis:

(For more detailed information, cf. Reference section.)

In [15]:
nice_stats = pa.NiceStatsProfileOrdinal(
    tests_profile=[
        (lambda profile: any([strategy for strategy in profile.analyzed_strategies.equilibria
                              if strategy.profile.condorcet_winners == strategy.winners]),
         'There exists a true equilibrium electing the CW'),
    ],
    tests_strategy=[
        (lambda strategy: strategy.profile.condorcet_winners == strategy.winners,
         'There exists an equilibrium that elects the CW')
    ],
    tests_strategy_dist=[
        (lambda strategy: strategy.profile.condorcet_winners == strategy.winners,
         'There exists an equilibrium that elects the CW')
    ],
   conditional_on=lambda profile: profile.is_profile_condorcet == 1.
)

Run the Monte-Carlo simulation:

In [16]:
# nice_stats.run(n_samples=10)

Synthetic view of the results:

In [17]:
# nice_stats.display_results()

Find an example for one of the tests:

In [18]:
# profile = nice_stats.find_example('There exists a true equilibrium electing the CW', False)
# profile

Draw a plot:

In [19]:
# nice_stats.plot_test_strategy('There exists an equilibrium that elects the CW')
# nice_stats.plot_cutoff('There exists a true equilibrium electing the CW',
#                        left='No true equilibrium', right='True equilibrium')
# plt.xlim(-0.05, 1.05)